## Problem.1

In [6]:
#import libraries
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#Parameters
gamma = 2.0
beta = 0.985**20
r = 1.025**20 - 1.0
l = np.array([0.8027, 1.0, 1.2457]) # Productivity levels
NL = 3 # Number of productivity levels
# Transition matrix
prob = np.array([
    [0.7451, 0.2528, 0.0021],
    [0.1360, 0.7281, 0.1360],
    [0.0021, 0.2528, 0.7451]
])

In [11]:
# Grids
a_l = 0.0
a_u = 4.0
NA = 100
a = np.linspace(a_l, a_u, NA)

In [12]:
# Utility function
def util(c, gamma):
    if c > 0:
        return (c**(1 - gamma)) / (1 - gamma)
    else:
        return -np.inf # Return negative infinity for non-positive consumption

In [ ]:
# Initialization
v = np.zeros((3, NA, NL)) # Value function for 3 periods
aplus = np.zeros((2, NA, NL)) # Savings policy function for young and middle age

In [ ]:
# Backward induction
# Period 3
for ia in range(NA):
    for il in range(NL):
        # In old age, there's no labor income, and all assets are consumed
        v[2, ia, il] = util((1.0 + r) * a[ia], gamma)

# Period 2
for il in range(NL): # Productivity in middle age
    for ia in range(NA): # Assets at the beginning of middle age
        reward = np.zeros(NA)
        for iap in range(NA): # Assets for the next period (old age)
            # Middle-aged labor income depends on the productivity shock
            # For simplicity, we assume middle-age income is the same as the productivity level 'y'
            # (as per the problem's focus on initial productivity)
            consumption = l[il] + (1.0 + r) * a[ia] - a[iap]
            if consumption > 0:
                reward[iap] = util(consumption, gamma) + beta * v[2, iap, 0] # v[2] is independent of 'l'
        aplus[1, ia, il] = a[np.argmax(reward)]
        v[1, ia, il] = np.max(reward)